In [15]:
import requests, time, random
from bs4 import BeautifulSoup
import re
import random
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, TimeoutException

# Get linkedin username and password form user:
print("Please enter the exact LinkedIn username you use to login (email/phone?):")
username_string = str(input()) 
print()
print("Please enter the exact LinkedIn password:")
password_string = str(input())
print()

#create a browser-specific (Google Chrome) web navigation simulator:
service = Service()
options = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=service, options=options)

# Navigate to the LinkedIn login page#
browser.get("https://www.linkedin.com/login")

# Enter your login credentials
elementID = browser.find_element(By.ID,'username')
elementID.send_keys(username_string)
elementID = browser.find_element(By.ID,'password')
elementID.send_keys(password_string)
elementID.submit()
time.sleep(5)

In [16]:
links=["https://www.linkedin.com/company/auxesisgroup/"]

In [17]:
len(links)

1

In [18]:
Name=[]
Title=[]
Followers=[]
Location=[]
URL=[]
Phone_no=[]
Company_size=[]
Domain=[]
Founded_year=[]
Specility=[]

for i in links:
    #Open Link
    browser.get(i)
    random_number = random.randint(2, 4)
    time.sleep(random_number)
        
    #Getting HTMML tags
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
        
    #Scrapping name
    name=browser.find_element(By.TAG_NAME,'h1').text.strip('\n...')
    name
    
    #Scrapping bio
    try:
        title = soup.find("p", {'class': 'org-top-card-summary__tagline org-top-card-improvement-summary__tagline'})
        bio = title.get_text().strip()
    except:
        bio='None'
    
    #Followers
    try:
        followers = soup.find_all("div", {'class': 'org-top-card-summary-info-list__info-item'})
        no_of_fol = followers[2].get_text().strip().replace(" followers", "")
        no_of_fol=int(no_of_fol.replace(",", ""))
    except:
        no_of_fol='None'
        
        
    #clicking about section
    try:
        browser.find_element(By.XPATH,'//h2[@class="text-heading-xlarge"]')
    except NoSuchElementException:
        about_button=browser.find_element(By.XPATH,'//a[@class="ember-view pv3 ph4 t-16 t-bold t-black--light org-page-navigation__item-anchor "]')
        about_button.click()                        
        time.sleep(2)  
    
    pag = browser.page_source
    soup2 = BeautifulSoup(pag, 'lxml')
    abt_sec=soup2.find('dl',{'class':'overflow-hidden'})
    dom=abt_sec.find_all('dd')
    dt_tags = abt_sec.find_all('dt',{'class':'mb1 text-heading-medium'})
    dd_tags = abt_sec.find_all('dd',{'class':'mb4 t-black--light text-body-medium'})
    
    #URL of Website
    try:
        web=abt_sec.find('a',{'class':'link-without-visited-state ember-view'})
        Url=web.get_text().strip()
    except:
        Url="None"


    # Create a dictionary to store the extracted information
    info_dict = {}
    
    # Iterate over <dt> tags and extract the information for website,phone, Industry, headquater, company size
    i=0
    x=1
    for dt_tag, dd_tag in zip(dt_tags, dd_tags):
        dt_text = dt_tag.text.strip()
        if (dt_text== "Website" or dt_text== "Phone" or dt_text== "Industry" or dt_text=="Headquarters"):
            dd_text = dd_tag.text.strip()
        
        if dt_text == "Company size":
            dd_tag = abt_sec.find('dd', {'class': 't-black--light text-body-medium mb1'})
            dd_text=dd_tag.get_text().strip().replace(" employees", "")
            
        if dt_text == "Headquarters":
            x=2
            dd_tag = dd_tags[i-1]
            dd_text=dd_tag.get_text().strip()
       
        info_dict[dt_text] = dd_text
        i=i+1
    
    #extract the information for founded and specialities
    for dt_tag in dt_tags:
        dt_text = dt_tag.text.strip()
        if dt_text == "Headquarters" and x==1:
            dd_tag = dd_tags[i-1]
            dd_text=dd_tag.get_text().strip()
            info_dict["Headquarters"] = dd_text
        if dt_text=="Specialties":
            dd_tag = dd_tags[i-1]
            dd_text=dd_tag.get_text().strip()
            info_dict[dt_text] = dd_text
            dd_tag = dd_tags[i-2]
            dd_text=dd_tag.get_text().strip()
            info_dict["Founded"] = dd_text
        if dt_text=="Founded":
            dd_tag = dd_tags[i-1]
            dd_text=dd_tag.get_text().strip()
            info_dict["Founded"] = dd_text
            
    # Handle case when fields is not present
    if "Specialties" not in info_dict:
        info_dict["Specialties"] = "None"
    if "Founded" not in info_dict:
        info_dict["Founded"] = "None"
    if "Headquarters" not in info_dict:
        info_dict["Headquarters"] = "None"
    if "Website" not in info_dict:
        info_dict["Website"] = "None"
    if "Company size" not in info_dict:
        info_dict["Company size"] = "None"        
    if "Industry" not in info_dict:
        info_dict["Industry"] = "None" 
        
        
    #appending for making dataset
    Name.append(name)
    Title.append(bio)
    Followers.append(no_of_fol)
    Location.append(info_dict["Headquarters"])
    URL.append(info_dict["Website"])
    #Phone_no.append(info_dict["Phone"])
    Company_size.append(info_dict["Company size"])
    Domain.append(info_dict["Industry"])
    Founded_year.append(info_dict["Founded"])
    Specility.append(info_dict["Specialties"])

    del info_dict
    random_number = random.randint(1, 10)
    time.sleep(random_number)

In [19]:
import pandas as pd
data={'Name':Name,'Title':Title,'Followers':Followers,'Website':URL,'Domain':Domain,'Company size':Company_size,'Location':Location,'Founded Year':Founded_year,'Specialization':Specility}
df=pd.DataFrame(data)
df.to_excel("Ozibook_DBPC.xlsx",index=False)
df

,Name,Title,Followers,Website,Domain,Company size,Location,Founded Year,Specialization
0,Auxesis Group,World's Leading Blockchain Infrastructure Company,3453,https://auxesisgroup.com,IT Services and IT Consulting,51-200,"Mumbai, Maharashtra",2015,"IT Business Consulting, Blockchain Technology,..."
